In [1]:
import pandas as pd
from pandas.api.types import CategoricalDtype
import numpy as np
import os
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import RobustScaler

In [2]:
from sklearn.preprocessing import LabelEncoder
pd.set_option('display.max_columns', None)

In [3]:
features = ['CASEID','SEX', 'RACE_NEW', 'ETHNICITY_HISPANIC', 'PRNCPTX', 'CPT', 'WORKRVU', 'AGE', 'ANESTHES', 'SURGSPEC', 'ELECTSURG', 'EMERGNCY', 'WNDCLAS', 'ASACLAS', 'DIABETES', 'SMOKE', 'DYSPNEA', 'FNSTATUS2', 'VENTILAT', 'HXCOPD', 'ASCITES', 'HXCHF', 'HYPERMED', 'RENAFAIL', 'DIALYSIS', 'DISCANCR', 'WNDINF', 'STEROID', 'WTLOSS', 'BLEEDDIS', 'TRANSFUS', 'PRSEPIS', 'HEIGHT', 'WEIGHT', 'PRSODM', 'PRBUN', 'PRCREAT', 'PRALBUM', 'PRBILI', 'PRSGOT', 'PRALKPH', 'PRWBC', 'PRHCT', 'PRPLATE', 'PRPTT', 'PRINR', 'PRPT', 'DOpertoD', 'OUPNEUMO', 'CDARREST', 'CDMI', 'SUPINFEC', 'WNDINFD', 'ORGSPCSSI', 'URNINFEC', 'OTHDVT', 'PULEMBOL', 'RENAINSF', 'OPRENAFL', 'DEHIS', 'REINTUB', 'FAILWEAN', 'CNSCVA', 'OTHSYSEP', 'OTHBLEED', 'OTHSESHOCK', 'SEPSHOCKPATOS', 'OTHERCPT1', 'OTHERCPT2', 'OTHERCPT3', 'OTHERCPT4', 'OTHERCPT5', 'OTHERCPT6', 'OTHERCPT7', 'OTHERCPT8', 'OTHERCPT9', 'OTHERCPT10', 'CONCPT1', 'CONCPT2', 'CONCPT3', 'CONCPT4', 'CONCPT5', 'CONCPT6', 'CONCPT7', 'CONCPT8', 'CONCPT9', 'CONCPT10', 'MORTPROB', 'MORBPROB', 'INOUT', 'TRANST', 'DPRNA', 'DPRBUN', 'DPRCREAT', 'DPRALBUM', 'DPRBILI', 'DPRSGOT', 'DPRALKPH', 'DPRWBC', 'DPRHCT', 'DPRPLATE', 'DPRPTT', 'DPRPT', 'DPRINR', 'OperYR', 'OPTIME', 'HtoODay', 'SSSIPATOS', 'DSSIPATOS', 'OSSIPATOS', 'PNAPATOS', 'VENTPATOS', 'UTIPATOS', 'SEPSISPATOS', 'SEPSHOCKPATOS', 'OPTIME', 'READMISSION1']

In [4]:
features = [x.upper() for x in features]

In [5]:
col_features = ['COL_STEROID', 'COL_MECH_BOWEL_PREP', 'COL_ORAL_ANTIBIOTIC', 'COL_CHEMO', 'COL_INDICATION', 'COL_ICD9_INDICATION', 'COL_EMERGENT', 'COL_ICD9_EMERGENT', 'COL_APPROACH', 'COL_ICD10_INDICATION', 'COL_ICD10_EMERGENT','COL_ANASTOMOTIC']

In [6]:
CPT_dict = {'OTHERCPT1': str, 'OTHERCPT2':str, 'OTHERCPT3':str, 'OTHERCPT4':str, 'OTHERCPT5':str, 'OTHERCPT6':str, 'OTHERCPT7':str, 'OTHERCPT8':str, 'OTHERCPT9':str, 'OTHERCPT10':str, 'CONCPT1':str, 'CONCPT2':str,'CONCPT3':str,'CONCPT4':str,'CONCPT5':str,'CONCPT6':str,'CONCPT7':str,'CONCPT8':str,'CONCPT9':str,'CONCPT10':str, 'REOPORCPT1':str, 'REOPOR2CPT1':str}

In [7]:
total = col_features + features

In [8]:
#Open csv file
data = pd.read_csv(r'/home/kchen/Documents/nsqip_raw/col/combined/col.csv', index_col="CASEID", usecols=total, dtype=CPT_dict, na_values = [-99, '-99', 'Unknown'], low_memory=False)
data.head()

,COL_STEROID,COL_MECH_BOWEL_PREP,COL_ORAL_ANTIBIOTIC,COL_CHEMO,COL_INDICATION,COL_ICD9_INDICATION,COL_EMERGENT,COL_ICD9_EMERGENT,COL_APPROACH,COL_ANASTOMOTIC,SEX,RACE_NEW,ETHNICITY_HISPANIC,PRNCPTX,CPT,WORKRVU,INOUT,TRANST,AGE,OPERYR,ANESTHES,SURGSPEC,ELECTSURG,HEIGHT,WEIGHT,DIABETES,SMOKE,DYSPNEA,FNSTATUS2,VENTILAT,HXCOPD,ASCITES,HXCHF,HYPERMED,RENAFAIL,DIALYSIS,DISCANCR,WNDINF,STEROID,WTLOSS,BLEEDDIS,TRANSFUS,PRSEPIS,DPRNA,DPRBUN,DPRCREAT,DPRALBUM,DPRBILI,DPRSGOT,DPRALKPH,DPRWBC,DPRHCT,DPRPLATE,DPRPTT,DPRPT,DPRINR,PRSODM,PRBUN,PRCREAT,PRALBUM,PRBILI,PRSGOT,PRALKPH,PRWBC,PRHCT,PRPLATE,PRPTT,PRINR,PRPT,OTHERCPT1,OTHERCPT2,OTHERCPT3,OTHERCPT4,OTHERCPT5,OTHERCPT6,OTHERCPT7,OTHERCPT8,OTHERCPT9,OTHERCPT10,CONCPT1,CONCPT2,CONCPT3,CONCPT4,CONCPT5,CONCPT6,CONCPT7,CONCPT8,CONCPT9,CONCPT10,EMERGNCY,WNDCLAS,ASACLAS,MORTPROB,MORBPROB,OPTIME,HTOODAY,SUPINFEC,SSSIPATOS,WNDINFD,DSSIPATOS,ORGSPCSSI,OSSIPATOS,DEHIS,OUPNEUMO,PNAPATOS,REINTUB,PULEMBOL,FAILWEAN,VENTPATOS,RENAINSF,OPRENAFL,URNINFEC,UTIPATOS,CNSCVA,CDARREST,CDMI,OTHBLEED,OTHDVT,OTHSYSEP,SEPSISPATOS,OTHSESHOCK,SEPSHOCKPATOS,DOPERTOD,READMISSION1,COL_ICD10_INDICATION,COL_ICD10_EMERGENT
CASEID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
6551967,No,Yes,No,No,Colon cancer,NaN,NaN,NaN,Laparoscopic w/ open assist,No definitive diagnosis of leak/leak related a...,male,Unknown/Not Reported,NaN,LAPAROSCOPY COLECTOMY PARTIAL W/ANASTOMOSIS,44204.0,26.42,Inpatient,Not transferred (admitted from home),64,2017.0,General,General Surgery,Yes,73.0,216.0,NON-INSULIN,No,No,Independent,No,No,No,No,Yes,No,No,No,No,No,No,No,No,None,7.0,76.0,7.0,NaN,NaN,NaN,NaN,7.0,7.0,7.0,NaN,NaN,83.0,139.0,8.4034,0.8032,NaN,NaN,NaN,NaN,7.1,38.0,190.0,NaN,0.9,NaN,44213,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,2-Clean/Contaminated,3-Severe Disturb,0.002584,0.082690,121.0,0.0,No Complication,No,No Complication,No,No Complication,No,No Complication,No Complication,No,No Complication,No Complication,No Complication,No,No Complication,No Complication,No Complication,No,No Complication,No Complication,No Complication,No Complication,No Complication,No Complication,No,No Complication,No,NaN,No,NaN,NaN
6552344,No,NaN,NaN,No,Other-Enter ICD-10 for diagnosis,NaN,NaN,NaN,Laparoscopic,No definitive diagnosis of leak/leak related a...,female,White,No,LAPS COLECTOMY PRTL W/RMVL TERMINAL ILEUM,44205.0,22.95,Inpatient,Not transferred (admitted from home),44,2017.0,General,General Surgery,Yes,64.0,185.0,NO,No,No,Independent,No,No,No,No,Yes,No,No,No,No,No,No,No,No,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.0,38.0,38.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0,43.4,373.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,2-Clean/Contaminated,2-Mild Disturb,0.000412,0.051675,64.0,0.0,No Complication,No,No Complication,No,No Complication,No,No Complication,No Complication,No,No Complication,No Complication,No Complication,No,No Complication,No Complication,No Complication,No,No Complication,No Complication,No Complication,No Complication,No Complication,No Complication,No,No Complication,No,NaN,No,NaN,NaN
6552431,No,Yes,Yes,No,Chronic diverticular disease,NaN,NaN,NaN,Laparoscopic,No definitive diagnosis of leak/leak related a...,female,White,No,LAPS COLECTOMY PRTL W/COLOPXTSTMY LW ANAST,44207.0,31.92,Inpatient,Not transferred (admitted from home),30,2017.0,General,General Surgery,Yes,66.0,180.0,NO,No,No,Independent,No,No,No,No,No,No,No,No,No,No,No,No,No,None,24.0,24.0,24.0,NaN,NaN,NaN,NaN,23.0,23.0,23.0,NaN,NaN,NaN,138.0,3.0000,0.6000,NaN,NaN,NaN,NaN,4.7,37.7,309.0,NaN,NaN,NaN,44213,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,3-Contaminated,2-Mild Disturb,0.000225,0.055749,99.0,0.0,No Complication,No,No Complication,No,No Complication,No,No Complication,No Complication,No,No Complication,No Complication,No Complication,No,No Complication,No Complica

In [9]:
#Create BMI column
lbs_to_kg_ratio = 0.453592
inch_to_meter_ratio = 0.0254

data['HEIGHT'] *= inch_to_meter_ratio
data['WEIGHT'] *= lbs_to_kg_ratio

data['BMI'] = data['WEIGHT']/(data['HEIGHT']**2)
print(min(data['BMI']))
print(max(data['BMI']))

9.097328448563147
139.05699614739444


In [10]:
print(data.columns.tolist())

['COL_STEROID', 'COL_MECH_BOWEL_PREP', 'COL_ORAL_ANTIBIOTIC', 'COL_CHEMO', 'COL_INDICATION', 'COL_ICD9_INDICATION', 'COL_EMERGENT', 'COL_ICD9_EMERGENT', 'COL_APPROACH', 'COL_ANASTOMOTIC', 'SEX', 'RACE_NEW', 'ETHNICITY_HISPANIC', 'PRNCPTX', 'CPT', 'WORKRVU', 'INOUT', 'TRANST', 'AGE', 'OPERYR', 'ANESTHES', 'SURGSPEC', 'ELECTSURG', 'HEIGHT', 'WEIGHT', 'DIABETES', 'SMOKE', 'DYSPNEA', 'FNSTATUS2', 'VENTILAT', 'HXCOPD', 'ASCITES', 'HXCHF', 'HYPERMED', 'RENAFAIL', 'DIALYSIS', 'DISCANCR', 'WNDINF', 'STEROID', 'WTLOSS', 'BLEEDDIS', 'TRANSFUS', 'PRSEPIS', 'DPRNA', 'DPRBUN', 'DPRCREAT', 'DPRALBUM', 'DPRBILI', 'DPRSGOT', 'DPRALKPH', 'DPRWBC', 'DPRHCT', 'DPRPLATE', 'DPRPTT', 'DPRPT', 'DPRINR', 'PRSODM', 'PRBUN', 'PRCREAT', 'PRALBUM', 'PRBILI', 'PRSGOT', 'PRALKPH', 'PRWBC', 'PRHCT', 'PRPLATE', 'PRPTT', 'PRINR', 'PRPT', 'OTHERCPT1', 'OTHERCPT2', 'OTHERCPT3', 'OTHERCPT4', 'OTHERCPT5', 'OTHERCPT6', 'OTHERCPT7', 'OTHERCPT8', 'OTHERCPT9', 'OTHERCPT10', 'CONCPT1', 'CONCPT2', 'CONCPT3', 'CONCPT4', 'CONCPT5

In [11]:
data.shape

(257913, 127)

In [12]:
#Remove unwanted CPT's from CPT
Ost_CPT = [44187, 44188, 44141, 44143, 44144, 44146, 44150, 44151, 44206, 44208, 44210,  44187, 44188, 44320, 44310]
for CPT in Ost_CPT:
    data.drop(data[data['CPT'] == CPT].index, inplace=True)
data.shape    

(206147, 127)

In [13]:
#Remove unwanted CPT's from OTHER and Concurrent CPT
for CPT in Ost_CPT:
    for i in range(1,11):
        data.drop(data[data['OTHERCPT%s' % i] == CPT].index, inplace=True)
        data.drop(data[data['CONCPT%s' % i] == CPT].index, inplace=True)

In [14]:
#Remove unwanted CPT's from CPT
Ost_CPT = ['44187', '44188', '44141', '44143', '44144', '44146', '44150', '44151', '44206', '44208', '44210',  '44187', '44188', '44320', '44310']
for CPT in Ost_CPT:
    data.drop(data[data['CPT'] == CPT].index, inplace=True)
data.shape

(206147, 127)

In [15]:
for CPT in Ost_CPT:
    for i in range(1,11):
        data.drop(data[data['OTHERCPT%s' % i] == CPT].index, inplace=True)
        data.drop(data[data['CONCPT%s' % i] == CPT].index, inplace=True)
data.shape

(194740, 127)

In [16]:
data.shape

(194740, 127)

In [17]:
data['COL_ANASTOMOTIC'].value_counts()

No definitive diagnosis of leak/leak related abscess       175438
No                                                          12347
Leak, treated w/ reoperation                                 3349
Leak, treated w/ interventional means                        1289
Leak, treated w/ non-interventional/non-operative means       818
Leak, no treatment intervention documented                    410
Yes-reoperation                                               299
Yes-percutaneous intervention                                 103
Yes-no intervention required                                   86
Name: COL_ANASTOMOTIC, dtype: int64

In [18]:
#Drop missing leak
le=LabelEncoder()
data['COL_ANASTOMOTIC']=le.fit_transform(data['COL_ANASTOMOTIC'])
data['COL_ANASTOMOTIC'].value_counts()

5    175438
4     12347
3      3349
1      1289
2       818
9       601
0       410
8       299
7       103
6        86
Name: COL_ANASTOMOTIC, dtype: int64

In [19]:
missing_leak = data[data['COL_ANASTOMOTIC'] == 6].index
data.drop(missing_leak, inplace=True)
data['COL_ANASTOMOTIC'].value_counts()

5    175438
4     12347
3      3349
1      1289
2       818
9       601
0       410
8       299
7       103
Name: COL_ANASTOMOTIC, dtype: int64

In [20]:
#Convert leak to binary classification
data.loc[data['COL_ANASTOMOTIC'] == 4, 'COL_ANASTOMOTIC'] = 0
data.loc[data['COL_ANASTOMOTIC'] == 5, 'COL_ANASTOMOTIC'] = 0
data['COL_ANASTOMOTIC'].value_counts()

0    188195
3      3349
1      1289
2       818
9       601
8       299
7       103
Name: COL_ANASTOMOTIC, dtype: int64

In [21]:
data.loc[data['COL_ANASTOMOTIC'] != 0, 'COL_ANASTOMOTIC'] = 1
data['COL_ANASTOMOTIC'].value_counts()

0    188195
1      6459
Name: COL_ANASTOMOTIC, dtype: int64

In [22]:
data['COL_ANASTOMOTIC'].value_counts(normalize=True)

0    0.966818
1    0.033182
Name: COL_ANASTOMOTIC, dtype: float64

In [23]:
data.loc[data['COL_ANASTOMOTIC'] == 0, 'COL_ANASTOMOTIC'] = False

data.loc[data['COL_ANASTOMOTIC'] != False, 'COL_ANASTOMOTIC'] = True
data['COL_ANASTOMOTIC'].value_counts(dropna=False)

False    188195
True       6459
Name: COL_ANASTOMOTIC, dtype: int64

In [24]:
data.groupby('OPERYR')['COL_ANASTOMOTIC'].value_counts(normalize=True)

OPERYR  COL_ANASTOMOTIC
2012.0  False              0.963781
        True               0.036219
2013.0  False              0.960123
        True               0.039877
2014.0  False              0.962477
        True               0.037523
2015.0  False              0.965220
        True               0.034780
2016.0  False              0.969048
        True               0.030952
2017.0  False              0.967045
        True               0.032955
2018.0  False              0.969908
        True               0.030092
2019.0  False              0.969932
        True               0.030068
Name: COL_ANASTOMOTIC, dtype: float64

In [25]:
#Cast ASA class as ordered categorical
cat_type = CategoricalDtype(categories=['None assigned','1-No Disturb','2-Mild Disturb','3-Severe Disturb','4-Life Threat','5-Moribund'], ordered=True)
data['ASACLAS'].astype(cat_type)


CASEID
6551967     3-Severe Disturb
6552344       2-Mild Disturb
6552431       2-Mild Disturb
6552941       2-Mild Disturb
6552982       2-Mild Disturb
                  ...       
10363817    3-Severe Disturb
10363850    3-Severe Disturb
10363894                 NaN
10363964    3-Severe Disturb
10363966      2-Mild Disturb
Name: ASACLAS, Length: 194654, dtype: category
Categories (6, object): ['None assigned' < '1-No Disturb' < '2-Mild Disturb' < '3-Severe Disturb' < '4-Life Threat' < '5-Moribund']

In [26]:
#Convert 90+ to 91 and AGE to int32
data.loc[data['AGE'] == '90+', 'AGE'] = 91
pd.to_numeric(data['AGE'], downcast='integer')

CASEID
6551967     64.0
6552344     44.0
6552431     30.0
6552941     45.0
6552982     50.0
            ... 
10363817    50.0
10363850    53.0
10363894     NaN
10363964    41.0
10363966    72.0
Name: AGE, Length: 194654, dtype: float64

In [27]:
cat_names = ['CPT', 'SEX', 'FNSTATUS2', 'ASACLAS', 'STEROID', 'ASCITES', 'PRSEPIS', 
             'VENTILAT', 'DIABETES', 'HYPERMED', 'HXCHF', 'DYSPNEA', 'SMOKE', 'HXCOPD', 
             'DIALYSIS', 'RENAFAIL', 'OPERYR', 
             'RACE_NEW', 'ETHNICITY_HISPANIC', 'WNDCLAS',
              'BLEEDDIS', 'WTLOSS', 'TRANSFUS', 'TRANST', 'OTHBLEED',
             'SSSIPATOS', 'DSSIPATOS', 'OSSIPATOS', 'PNAPATOS', 'VENTPATOS', 'UTIPATOS', 'SEPSISPATOS', 'SEPSHOCKPATOS',
             'COL_INDICATION', 'COL_EMERGENT', 'COL_STEROID', 'COL_MECH_BOWEL_PREP', 'COL_ORAL_ANTIBIOTIC', 'COL_CHEMO', 'COL_APPROACH'
            ]
cont_names = ['HEIGHT', 'WEIGHT', 'BMI', 'AGE', 'PRSODM', 'PRBUN', 'PRCREAT', 'PRALBUM', 'PRBILI', 'PRALKPH', 'PRWBC', 'PRHCT', 'PRPLATE', 'PRPTT', 'PRINR', 'PRPT']
y_name = ['COL_ANASTOMOTIC']

In [28]:
features = cat_names + cont_names + y_name

In [29]:
data = data[features]

In [30]:
data.loc[data['COL_APPROACH'] == 'SILS', 'COL_APPROACH'] = 'Laparoscopic'
data.loc[data['COL_APPROACH'] == 'SILS w/ open assist', 'COL_APPROACH'] = 'Laparoscopic w/ open assist'
data.loc[data['COL_APPROACH'] == 'SILS w/ unplanned conversion to open', 'COL_APPROACH'] = 'Laparoscopic w/ unplanned conversion to open'

data.loc[data['COL_APPROACH'] == 'Endoscopic', 'COL_APPROACH'] = 'Laparoscopic'
data.loc[data['COL_APPROACH'] == 'Endoscopic w/ open assist', 'COL_APPROACH'] = 'Laparoscopic w/ open assist'
data.loc[data['COL_APPROACH'] == 'Endoscopic w/ unplanned conversion to open', 'COL_APPROACH'] = 'Laparoscopic w/ unplanned conversion to open'

data.loc[data['COL_APPROACH'] == 'NOTES', 'COL_APPROACH'] = 'Laparoscopic'
data.loc[data['COL_APPROACH'] == 'NOTES w/ open assist', 'COL_APPROACH'] = 'Laparoscopic w/ open assist'
data.loc[data['COL_APPROACH'] == 'NOTES w/ unplanned conversion to open', 'COL_APPROACH'] = 'Laparoscopic w/ unplanned conversion to open'

data.loc[data['COL_APPROACH'] == 'Other MIS approach', 'COL_APPROACH'] = 'Laparoscopic'
data.loc[data['COL_APPROACH'] == 'Other MIS approach w/ open assist', 'COL_APPROACH'] = 'Laparoscopic w/ open assist'
data.loc[data['COL_APPROACH'] == 'Other MIS approach w/ unplanned conversion to open', 'COL_APPROACH'] = 'Laparoscopic w/ unplanned conversion to open'

data.loc[data['COL_APPROACH'] == 'Hybrid', 'COL_APPROACH'] = 'Laparoscopic'
data.loc[data['COL_APPROACH'] == 'Hybrid w/ open assist', 'COL_APPROACH'] = 'Laparoscopic w/ open assist'
data.loc[data['COL_APPROACH'] == 'Hybrid w/ unplanned conversion to open', 'COL_APPROACH'] = 'Laparoscopic w/ unplanned conversion to open'

data.loc[data['COL_APPROACH'] == 'Laparoscopic w/ unplanned conversion to Open', 'COL_APPROACH'] = 'Laparoscopic w/ unplanned conversion to open'
data.loc[data['COL_APPROACH'] == 'Laparoscopic Hand Assisted', 'COL_APPROACH'] = 'Laparoscopic w/ open assist'
data.loc[data['COL_APPROACH'] == 'Open', 'COL_APPROACH'] = 'Open (planned)'

data['COL_APPROACH'].value_counts()

Laparoscopic                                    61174
Open (planned)                                  54286
Laparoscopic w/ open assist                     46589
Laparoscopic w/ unplanned conversion to open    13652
Robotic                                         11517
Robotic w/ open assist                           6293
Robotic w/ unplanned conversion to open           958
Other                                             127
Name: COL_APPROACH, dtype: int64

In [31]:
data.shape

(194654, 57)

num_cols = ['AGE','ANETIME','HEIGHT','WEIGHT','BMI','PRSODM','PRBUN','PRCREAT','PRALBUM','PRBILI','PRSGOT','PRALKPH','PRWBC','PRHCT','PRPLATE','PRPTT','PRINR','PRPT','OPTIME', 'HTOODAY']
cat_cols = list(set(total) - set(num_cols) - set(outcomes) - set(['CASEID','ETHNICITY_HISPANIC','RACE_NEW']))
cat_cols.append('RACE')

In [32]:
data_imputed = data.copy()

In [33]:
for col in cat_names:
    data_imputed[col].fillna(value='Unknown', inplace=True)

In [34]:
col_missing = data_imputed.isnull().sum()/data.shape[0]
col_missing = col_missing.sort_values(ascending=False)
col_missing = col_missing[col_missing > 0]
(pd.DataFrame(col_missing)).head(20)

,0
PRPT,0.992756
PRPTT,0.661322
PRINR,0.537877
PRBILI,0.307705
PRALKPH,0.307700
PRALBUM,0.303508
PRBUN,0.113319
PRSODM,0.080697
PRCREAT,0.073515
PRPLATE,0.063240


In [35]:
drop = ['PRPT', 'PRINR','PRPTT', 'PRALKPH','PRBILI','PRALBUM']
data_imputed.drop(drop, axis=1, inplace=True)

In [36]:
cont_names = list(set(cont_names) - set(drop))

In [37]:
num_imputer = SimpleImputer(strategy = 'median')

# Imputation for numeric:
for ncol in cont_names:
    data_imputed[ncol] = num_imputer.fit_transform(data_imputed[ncol].values.reshape(-1, 1))

In [38]:
for col in cont_names:
    data_imputed[col] = RobustScaler().fit_transform(data_imputed[col].values.reshape(-1,1))

In [39]:
le=LabelEncoder()
for col in cat_names:
        data_imputed[col] = le.fit_transform(data_imputed[col].astype(str))

In [ ]:
data_imputed.loc[data_imputed['COL_ANASTOMOTIC'] == True, 'COL_ANASTOMOTIC'] = 1
data_imputed.loc[data_imputed['COL_ANASTOMOTIC'] == False, 'COL_ANASTOMOTIC'] = 0

In [ ]:
data_imputed.to_csv(r'/home/kchen/Documents/leak/new_leak/clean_col.csv')

data.reset_index(inplace=True)

list = ['CASEID', 'CPT', 'SEX', 'FNSTATUS2', 'ASACLAS', 'STEROID', 'ASCITES', 'PRSEPIS', 'VENTILAT', 'DIABETES', 'HYPERMED', 'HXCHF', 'DYSPNEA', 'SMOKE', 'HXCOPD', 'DIALYSIS', 'RENAFAIL', 'OPERYR', 'RACE_NEW', 'ETHNICITY_HISPANIC', 'WNDCLAS', 'BLEEDDIS', 'WTLOSS', 'TRANSFUS', 'TRANST', 'SSSIPATOS', 'DSSIPATOS', 'OSSIPATOS', 'PNAPATOS', 'VENTPATOS', 'UTIPATOS', 'SEPSISPATOS', 'SEPSHOCKPATOS', 'SEPSHOCKPATOS', 'COL_INDICATION', 'COL_EMERGENT', 'COL_STEROID', 'COL_MECH_BOWEL_PREP', 'COL_ORAL_ANTIBIOTIC', 'COL_CHEMO', 'HEIGHT', 'WEIGHT', 'AGE', 'PRSODM', 'PRBUN', 'PRCREAT', 'PRALBUM', 'PRBILI', 'PRALKPH', 'PRWBC', 'PRHCT', 'PRPLATE', 'PRPTT', 'PRINR', 'PRPT', 'SEX', 'RACE_NEW', 'ETHNICITY_HISPANIC', 'PRNCPTX', 'CPT', 'WORKRVU', 'AGE', 'ANESTHES', 'SURGSPEC', 'ELECTSURG', 'EMERGNCY', 'WNDCLAS', 'ASACLAS', 'DIABETES', 'SMOKE', 'DYSPNEA', 'FNSTATUS2', 'VENTILAT', 'HXCOPD', 'ASCITES', 'HXCHF', 'HYPERMED', 'RENAFAIL', 'DIALYSIS', 'DISCANCR', 'WNDINF', 'STEROID', 'WTLOSS', 'BLEEDDIS', 'TRANSFUS', 'PRSEPIS', 'HEIGHT', 'WEIGHT', 'PRSODM', 'PRBUN', 'PRCREAT', 'PRALBUM', 'PRBILI', 'PRSGOT', 'PRALKPH', 'PRWBC', 'PRHCT', 'PRPLATE', 'PRPTT', 'PRINR', 'PRPT', 'DOPERTOD', 'OUPNEUMO', 'CDARREST', 'CDMI', 'SUPINFEC', 'WNDINFD', 'ORGSPCSSI', 'URNINFEC', 'OTHDVT', 'PULEMBOL', 'RENAINSF', 'OPRENAFL', 'DEHIS', 'REINTUB', 'FAILWEAN', 'CNSCVA', 'OTHSYSEP', 'OTHBLEED', 'OTHSESHOCK', 'SEPSHOCKPATOS', 'SEPSHOCKPATOS', 'OTHERCPT1', 'OTHERCPT2', 'OTHERCPT3', 'OTHERCPT4', 'OTHERCPT5', 'OTHERCPT6', 'OTHERCPT7', 'OTHERCPT8', 'OTHERCPT9', 'OTHERCPT10', 'CONCPT1', 'CONCPT2', 'CONCPT3', 'CONCPT4', 'CONCPT5', 'CONCPT6', 'CONCPT7', 'CONCPT8', 'CONCPT9', 'CONCPT10', 'MORTPROB', 'MORBPROB', 'INOUT', 'TRANST', 'DPRNA', 'DPRBUN', 'DPRCREAT', 'DPRALBUM', 'DPRBILI', 'DPRSGOT', 'DPRALKPH', 'DPRWBC', 'DPRHCT', 'DPRPLATE', 'DPRPTT', 'DPRPT', 'DPRINR', 'OPERYR', 'OPTIME', 'OPTIME', 'HTOODAY', 'SSSIPATOS', 'DSSIPATOS', 'OSSIPATOS', 'PNAPATOS', 'VENTPATOS', 'UTIPATOS', 'SEPSISPATOS', 'SEPSHOCKPATOS', 'SEPSHOCKPATOS', 'OPTIME', 'OPTIME', 'READMISSION1']

len(list)